In [81]:
import pandas as pd
import numpy as np
import numpy.random as random
from datetime import datetime
import random
import plotly.express as px
from google.colab import drive


In [27]:
#This brings in the normalized counts for all the individuals with Trisomy 21 in the Human Trisome Project. These are not the real count data but are similar to reall count data.
#df=pd.read_csv('/content/drive/MyDrive/normcounts.csv')
gene_exp_url="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/value_expression.csv"
gene_exp_df=pd.read_csv(gene_exp_url, index_col=0)

#this brings in the medical disorders for all the individuals with Trisomy 21 in the Human Trisome Project
comorbid_url = "https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/comorbid_file.csv"
comorbid_df = pd.read_csv(comorbid_url, index_col=0)

#this brings in random names to use
namefile = "https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/namelist.txt"
namesdf = pd.read_csv(namefile, index_col=0, names=["name"])



In [3]:
samplename="Patient"

# set paramaters

In [4]:
total_people = gene_exp_df.shape[0]


# Plot the real gene expression data

Removed the genes with a mean expression of 0.

In [5]:
genenames = [colname for colname in gene_exp_df.columns if colname!=samplename]


gene_exp_df_2 = gene_exp_df[genenames]
gene_exp_df_2 = gene_exp_df_2.T
gene_exp_df_2["mean"] = gene_exp_df_2.mean(axis=1)
gene_exp_df_2["std"] = gene_exp_df_2.std(axis=1)
gene_exp_df_2 = gene_exp_df_2[["mean", "std"]]
gene_exp_df_2 = gene_exp_df_2[gene_exp_df_2["mean"]!=0]
gene_exp_df_2["log_mean"] = np.log(gene_exp_df_2["mean"])
gene_exp_df_2["log_std"] = np.log(gene_exp_df_2["std"])
gene_exp_df_2

,mean,std,log_mean,log_std
ENSG00000141956,640.959623,163.052063,6.462966,5.094070
ENSG00000141959,8209.914576,2060.756148,9.013098,7.630828
ENSG00000142149,12.062413,9.275331,2.490094,2.227358
ENSG00000142156,122.892891,66.412714,4.811313,4.195889
ENSG00000142166,8054.859123,2526.434740,8.994031,7.834564
...,...,...,...,...
ENSG00000281383,27.161210,23.784712,3.301790,3.169043
ENSG00000281420,0.165025,0.555985,-1.801660,-0.587014
ENSG00000281903,7.866034,4.785761,2.062554,1.565645
ENSG00000283051,0.387181,0.980281,-0.948863,-0.019916


In [6]:
fig = px.scatter(gene_exp_df_2, x="log_mean", y="log_std")
fig.show()


# Plot the read comorbidity data

In [7]:
comorbidnames = [colname for colname in comorbid_df.columns if colname!=samplename]

comorbid_df
comorbid_df2 = comorbid_df[comorbidnames].T
comorbid_df2["sum"] = comorbid_df2.sum(axis=1)
comorbid_df2 = comorbid_df2[["sum"]]


In [8]:
fig = px.violin(comorbid_df2, y="sum")
fig.show()

In [9]:
comorbid_df2 = comorbid_df2[comorbid_df2["sum"]!=total_people]
comorbid_df2 = comorbid_df2[comorbid_df2["sum"]!=0]
comorbid_df2

,sum
congenital_heart_disease,141
hypothyroidism,135
sleep_apnea_syndrome,141
obstructive_sleep_apnea_syndrome,86
atrial_septal_defect,82
...,...
congenital_pulmonary_venous_return_anomaly,1
juvenile_idiopathic_arthritis,1
schizophrenia,1
separation_anxiety_disorder,1


In [ ]:
fig = px.violin(comorbid_df2, y="sum")
fig.show()

# code that makes the simulated data

## pick names for new people

In [ ]:
# Select a random sample of names from namesdf
random_names = namesdf.sample(n=total_people, replace=False)

random_names["name"] = random_names.index


#Set simulated parmaters

In [92]:
simulate_n_genes= 10
ncormobid_steps=25
biasdirs=["top", "bottom"]
topxpercentpeoplelist=[0.1, 0.25]
percentwithcomorbidintoppercentpeoplelist=[0.75,0.5,0.25]
ncomorbid_people_min = comorbid_df2["sum"].min()
ncomorbid_people_max =comorbid_df2["sum"].max()


# Pick genes that we will then simulate data for

## First simulate the gene expression based on real genes

In [94]:
# Cut the data from the 'mean' column into simulate_n_genes bins with equal number of rows
gene_exp_df_2['exp_group'] = pd.qcut(gene_exp_df_2['mean'], q=simulate_n_genes, labels=False)


# Create an empty DataFrame to store the randomly selected rows
simulated_gene_exp_df = pd.DataFrame()

# Iterate through each unique group in 'exp_group'
for group in gene_exp_df_2['exp_group'].unique():
  # Select rows belonging to the current group
  rows_in_group = gene_exp_df_2[gene_exp_df_2['exp_group'] == group]
  # Randomly select one row from the group
  if not rows_in_group.empty:
    random_row = rows_in_group.sample(n=1)
    # Append the randomly selected row to the DataFrame
    simulated_gene_exp_df = pd.concat([simulated_gene_exp_df, random_row])

# Display the DataFrame with randomly selected rows
print(simulated_gene_exp_df)

                        mean          std  log_mean   log_std  bins  exp_group
ENSG00000141956   640.959623   163.052063  6.462966  5.094070     8          8
ENSG00000141959  8209.914576  2060.756148  9.013098  7.630828     9          9
ENSG00000142149    12.062413     9.275331  2.490094  2.227358     6          6
ENSG00000142156   122.892891    66.412714  4.811313  4.195889     7          7
ENSG00000142166  8054.859123  2526.434740  8.994031  7.834564     9          9
...                      ...          ...       ...       ...   ...        ...
ENSG00000281383    27.161210    23.784712  3.301790  3.169043     7          7
ENSG00000281420     0.165025     0.555985 -1.801660 -0.587014     2          2
ENSG00000281903     7.866034     4.785761  2.062554  1.565645     6          6
ENSG00000283051     0.387181     0.980281 -0.948863 -0.019916     3          3
ENSG00000284550     0.003642     0.057926 -5.615276 -2.848581     0          0

[657 rows x 6 columns]


In [21]:
def variableexp(total_people, mean_exp, std_exp):
  arr = np.random.normal(mean_exp, std_exp, total_people)
  return arr

def generate_gene_exp(simulated_gene_exp_df):
  simulated_gene_exp_df["exp_array"] =  simulated_gene_exp_df.apply(lambda row: variableexp(total_people, row["mean"], row["std"]), axis=1)
  return simulated_gene_exp_df

In [22]:
simulated_gene_exp_df = generate_gene_exp(simulated_gene_exp_df)
simulated_gene_exp_df["sim_gene_name"] = "simulated_based_on_"+simulated_gene_exp_df.index
simulated_gene_exp_df = [random_names["name"].to_list() for i in range(simulated_gene_exp_df.shape[0])]
simulated_gene_exp_df["names"] = repeat_random_names
simulated_gene_exp_df = simulated_gene_exp_df[["names", "exp_array", "sim_gene_name"]].copy()

In [90]:
simulated_gene_exp_df_long = simulated_gene_exp_df.explode(["names", "exp_array"])
simulated_gene_exp_df_long

,names,exp_array,sim_gene_name
ENSG00000275993,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[485.4289275878412, 373.53609319572104, 693.41...",simulated_based_on_ENSG00000275993
ENSG00000280800,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[6121.677804768011, 621.2682707840886, -2073.3...",simulated_based_on_ENSG00000280800
ENSG00000280594,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[2.1709356211669917, -6.855039611738645, 19.66...",simulated_based_on_ENSG00000280594
ENSG00000276529,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[114.42858185170034, 87.14494294579404, 62.956...",simulated_based_on_ENSG00000276529
ENSG00000226054,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[-0.5656560775105408, -0.08495518874617175, -0...",simulated_based_on_ENSG00000226054
ENSG00000231355,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[0.1397946142674118, 3.915081320880714, 0.3584...",simulated_based_on_ENSG00000231355
ENSG00000230870,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[2.239440142405969, 0.8786264500822485, 5.4608...",simulated_based_on_ENSG00000230870
ENSG00000251778,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[-0.07895245880611376, 0.08803401123690419, -0...",simulated_based_on_ENSG00000251778
ENSG00000222042,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[0.6419122036393116, -0.04709363236482379, -0....",simulated_based_on_ENSG00000222042
ENSG00000230379,"[Amabel, Georgia, Eadith, Roxine, Fredia, Mari...","[-0.08109830184624513, 0.002602374900001623, 0...",simulated_based_on_ENSG00000230379


,names,exp_array,sim_gene_name
ENSG00000275993,Amabel,485.428928,simulated_based_on_ENSG00000275993
ENSG00000275993,Georgia,373.536093,simulated_based_on_ENSG00000275993
ENSG00000275993,Eadith,693.418316,simulated_based_on_ENSG00000275993
ENSG00000275993,Roxine,194.634579,simulated_based_on_ENSG00000275993
ENSG00000275993,Fredia,269.620154,simulated_based_on_ENSG00000275993
...,...,...,...
ENSG00000230379,Wallie,-0.193862,simulated_based_on_ENSG00000230379
ENSG00000230379,Brigida,0.266348,simulated_based_on_ENSG00000230379
ENSG00000230379,Doralia,-0.224896,simulated_based_on_ENSG00000230379
ENSG00000230379,Norah,-0.025047,simulated_based_on_ENSG00000230379


In [76]:
# prompt: pivot the dataframe random_rows_df_long so the column sim_gene_name is the new column names, the "names" column is the new row index and the values are in the dataframe

pivot_df = simulated_gene_exp_df_long.pivot(index='names', columns='sim_gene_name', values='exp_array')
pivot_df


sim_gene_name,simulated_based_on_ENSG00000222042,simulated_based_on_ENSG00000226054,simulated_based_on_ENSG00000230379,simulated_based_on_ENSG00000230870,simulated_based_on_ENSG00000231355,simulated_based_on_ENSG00000251778,simulated_based_on_ENSG00000275993,simulated_based_on_ENSG00000276529,simulated_based_on_ENSG00000280594,simulated_based_on_ENSG00000280800
names,,,,,,,,,,
Abbey,-0.366392,0.739591,-0.026557,9.270048,2.533378,-0.092275,650.654044,71.32166,13.335834,7971.699064
Adela,0.132886,0.070177,-0.114314,3.093491,2.72466,-0.005252,456.008542,119.582464,2.272846,1760.961412
Adiana,0.525249,0.287662,0.049612,6.194167,2.759613,-0.018695,206.230839,60.369642,6.797613,8685.701392
Aggie,-0.21571,2.244576,0.127855,7.956277,2.962732,-0.003323,480.174042,30.218599,9.963214,6685.410961
Ainsley,0.475153,0.352449,-0.295696,11.014179,-2.780056,-0.112835,183.570894,82.12701,13.251781,-2984.258324
...,...,...,...,...,...,...,...,...,...,...
Yolanda,-0.141433,0.207957,0.143574,0.453582,1.375848,0.019391,840.85686,79.946703,7.817333,4797.461313
Yolande,0.829867,0.405748,-0.218507,-3.434213,-0.603855,0.009057,625.148544,116.805415,1.023709,-2363.952062
Yolane,-0.319411,0.349377,-0.114457,5.374319,1.112093,0.059434,334.801404,141.161233,11.975738,4366.410001


In [78]:
# prompt: remove the index name from pivot_df

pivot_df.index.name = None
pivot_df

sim_gene_name,simulated_based_on_ENSG00000222042,simulated_based_on_ENSG00000226054,simulated_based_on_ENSG00000230379,simulated_based_on_ENSG00000230870,simulated_based_on_ENSG00000231355,simulated_based_on_ENSG00000251778,simulated_based_on_ENSG00000275993,simulated_based_on_ENSG00000276529,simulated_based_on_ENSG00000280594,simulated_based_on_ENSG00000280800
Abbey,-0.366392,0.739591,-0.026557,9.270048,2.533378,-0.092275,650.654044,71.32166,13.335834,7971.699064
Adela,0.132886,0.070177,-0.114314,3.093491,2.72466,-0.005252,456.008542,119.582464,2.272846,1760.961412
Adiana,0.525249,0.287662,0.049612,6.194167,2.759613,-0.018695,206.230839,60.369642,6.797613,8685.701392
Aggie,-0.21571,2.244576,0.127855,7.956277,2.962732,-0.003323,480.174042,30.218599,9.963214,6685.410961
Ainsley,0.475153,0.352449,-0.295696,11.014179,-2.780056,-0.112835,183.570894,82.12701,13.251781,-2984.258324
...,...,...,...,...,...,...,...,...,...,...
Yolanda,-0.141433,0.207957,0.143574,0.453582,1.375848,0.019391,840.85686,79.946703,7.817333,4797.461313
Yolande,0.829867,0.405748,-0.218507,-3.434213,-0.603855,0.009057,625.148544,116.805415,1.023709,-2363.952062
Yolane,-0.319411,0.349377,-0.114457,5.374319,1.112093,0.059434,334.801404,141.161233,11.975738,4366.410001
Zarah,-0.487143,-0.642195,0.311893,4.875746,3.66644,0.021941,660.021449,79.992784,5.01631,3442.587063


In [86]:
drive.mount('/content/drive')

Mounted at /content/drive


In [87]:
outdir="/content/drive/MyDrive/temp/"
pivot_df.to_csv(outdir+"simulated_gene_exp.csv")

In [ ]:


def makeagreatcomorbidlist(ncomorbid_people, topxpercentpeople=0.25, percentwithcomorbidintoppercentpeople = 0.5, biasdir="top"):
    ntopxpercentpeople=int(round(total_people*topxpercentpeople))
    print ("number of people in the --top--", ntopxpercentpeople)
    topquarter = list(np.repeat(0,ntopxpercentpeople))
    ncomorbid_people_top = percentwithcomorbidintoppercentpeople*ncomorbid_people
    ncomorbid_people_top = int(ncomorbid_people_top)
    print(ncomorbid_people_top, " in "+biasdir+"  ", ntopxpercentpeople, "total comorbid ", ncomorbid_people)
    if ncomorbid_people_top>ntopxpercentpeople:
        ncomorbid_people_top=ntopxpercentpeople
        print("nevermind", ncomorbid_people_top, " in "+biasdir+"  ", ntopxpercentpeople, "total comorbid ", ncomorbid_people)
    while sum(topquarter)<ncomorbid_people_top:
        indexsthatare0 = [i for i, val in enumerate(topquarter) if val==0]
        if len(indexsthatare0)==0:
            print ("ran out of 0")
            ncomorbid_people_top = sum(topquarter)
        else:
            rindex =np.random.choice(indexsthatare0)
            topquarter[rindex] = 1
    print ("finished the top", biasdir+"_quarter sum", sum(topquarter))
    per_in_top_q=sum(topquarter)/len(topquarter)
    print(biasdir+"_quarter percent", per_in_top_q)
    topxpeople = len(topquarter)
    topxpeoplencomorbid = sum(topquarter)
    great_comorbidity_binary = topquarter+list(np.repeat(0,total_people-ntopxpercentpeople))
    print ("great_comorbidity_binary sum", sum(great_comorbidity_binary))
    while sum(great_comorbidity_binary)<=ncomorbid_people:
        indexsthatare0 = [i for i, val in enumerate(great_comorbidity_binary) if val==0]
        rindex =np.random.choice(indexsthatare0)
        if rindex>len(topquarter):
            great_comorbidity_binary[rindex] = 1
    print ("great_comorbidity_binary", sum(great_comorbidity_binary), len(great_comorbidity_binary))
    #great_comorbidity_binary = np.concatenate((np.repeat(1, ncomorbid_people),np.repeat(0, nnotcomorbid_people)), axis=None)
    great_comorbidity_binary = np.array(great_comorbidity_binary)
    return great_comorbidity_binary, topxpeople, topxpeoplencomorbid


def makegeneexpandcomorbidfile(mean_exp_list, mean_minus_stdev_list, meanplus_stdev_list):
    genexpdf = namesdf.sample(total_people)
    genexpdf = variablestdgeneexp(genexpdf, mean_exp_list, mean_minus_stdev_list, meanplus_stdev_list)
    comorbid_df = genexpdf[["Patient"]].copy()
    genexpdf = genexpdf.reset_index(drop=True)
    comorbid_df = comorbid_df.reset_index(drop=True)
    genenames = [colname for colname in genexpdf.columns if colname!="Patient"]
    for sortgene in genenames:
        for biasdir in biasdirs:
            genexpdf, comorbid_df = sortbyagene(genexpdf, comorbid_df, sortgene = sortgene, biasdir= biasdir)
            comorbid_df = variable_comorbiddf(comorbid_df, sortgene, ncomorbid_people_min =ncomorbid_people_min, ncomorbid_people_max =ncomorbid_people_max, topxpercentpeoplelist=topxpercentpeoplelist, percentwithcomorbidintoppercentpeoplelist=percentwithcomorbidintoppercentpeoplelist, biasdir=biasdir)
    return genexpdf, comorbid_df

def sortbyagene(genexpdf, comorbid_df, sortgene = "gene_1", biasdir="top"):

    if biasdir=="top":
        genexpdf = genexpdf.sort_values(sortgene,ascending=False)
    else:
        genexpdf = genexpdf.sort_values(sortgene)
    comorbid_df = comorbid_df.reindex(index=genexpdf.index)
    return genexpdf, comorbid_df

def variablestdgeneexp(genexpdf, mean_exp, min_stdev, max_stdev):
    for std_dev in range(min_stdev, max_stdev):
        genename = "gene"+str(std_dev)
        genexpdf[genename] = np.random.normal(loc=mean_exp, scale=std_dev, size=total_people)
    return genexpdf

def variable_comorbiddf(comorbid_df, sortgene, ncomorbid_people_min, ncomorbid_people_max, topxpercentpeoplelist, percentwithcomorbidintoppercentpeoplelist, biasdir):
    for ncomorbid_people in range(ncomorbid_people_min, ncomorbid_people_max, ncormobid_steps):
        ncomorbid_people = int(ncomorbid_people)
        print ("on ncomorbid_people",ncomorbid_people )
        whichrandom = 0
        nnotcomorbid_people = total_people-ncomorbid_people
        comorbid_df["perfect_"+biasdir+"_comorbidity_"+str(sortgene)+"_ncomorbid"+str(ncomorbid_people)] = np.concatenate((np.repeat(1, ncomorbid_people),np.repeat(0, nnotcomorbid_people)), axis=None)
        for percentwithcomorbidintoppercentpeople in percentwithcomorbidintoppercentpeoplelist:
            for topxpercentpeople in topxpercentpeoplelist:

                print ("on ",percentwithcomorbidintoppercentpeople, topxpercentpeople)
                comorbidlist, topxpeople, topxpeoplencomorbid = makeagreatcomorbidlist(ncomorbid_people, topxpercentpeople=topxpercentpeople, percentwithcomorbidintoppercentpeople = percentwithcomorbidintoppercentpeople)
                if topxpeople<topxpeoplencomorbid:
                    topxpeoplencomorbid=topxpeople
                newcolname = "great_comorbidity_"+str(sortgene)+"_ncomorbid"+str(ncomorbid_people)+"_"+biasdir+str(topxpeople)+"_topcomorbid"+str(topxpeoplencomorbid)
                comorbid_df[newcolname] = comorbidlist
                whichrandom = whichrandom+1
                comorbid_df["random_comorbidity_ncomorbid_"+str(ncomorbid_people)+"random_"+str(whichrandom)] = np.random.permutation(comorbid_df[newcolname])
    return comorbid_df


def makeandsave():
    now = datetime.now()
    date_time = now.strftime("%m_%d_%Y_%H_%M_%S")
    print(date_time)
    #I am here mean_exp_list, mean_minus_stdev_list, meanplus_stdev_list
    genexpdf, comorbid_df = makegeneexpandcomorbidfile(mean_exp, min_stdev, max_stdev)
    biasdir="both"
    genexpdf.to_csv(outdir+"simulated_geneexp_bias"+biasdir+date_time+".csv")
    comorbid_df.to_csv(outdir+"simulated_comorbid_bias_"+biasdir+date_time+".csv")
    wf = open(outdir+"simulated_paramaters_bias_"+biasdir+date_time+".txt", "w")
    wf.write("total_people="+str(total_people)+"\n")
    wf.write("mean_exp="+str(mean_exp)+"\n")
    wf.write("min_stdev="+str(max_stdev)+"\n")
    wf.write("ncomorbid_people_min="+str(ncomorbid_people_min)+"\n")
    wf.write("ncomorbid_people_max="+str(ncomorbid_people_max)+"\n")
    wf.write("ncormobid_steps="+str(ncormobid_steps)+"\n")
    wf.write("biasdir="+biasdir+"\n")
    wf.write("topxpercentpeoplelist="+",".join(map(str,topxpercentpeoplelist))+"\n")
    wf.write("percentwithcomorbidintoppercentpeoplelist="+",".join(map(str,percentwithcomorbidintoppercentpeoplelist))+"\n")
    wf.close()

# make and save an output

In [ ]:
makeandsave()

09_20_2024_14_30_23
on ncomorbid_people 10
on  0.75 0.1
number of people in the --top-- 25
7  in top   25 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.28
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.75 0.25
number of people in the --top-- 64
7  in top   64 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.109375
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.5 0.1
number of people in the --top-- 25
5  in top   25 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.2
great_comorbidity_binary sum 5
great_comorbidity_binary 11 254
on  0.5 0.25
number of people in the --top-- 64
5  in top   64 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.078125
great_comorbidity_binary sum 5
great_comorbidity_binary 11 254
on  0.25 0.1
number of people in the --top-- 25
2  in top   25 total comorbid  10
finished the top top_quarter sum 2
top_quarter p

/tmp/ipykernel_97293/2603949995.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  comorbid_df[newcolname] = comorbidlist
/tmp/ipykernel_97293/2603949995.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  comorbid_df["random_comorbidity_ncomorbid_"+str(ncomorbid_people)+"random_"+str(whichrandom)] = np.random.permutation(comorbid_df[newcolname])
/tmp/ipykernel_97293/2603949995.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

on ncomorbid_people 10
on  0.75 0.1
number of people in the --top-- 25
7  in top   25 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.28
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.75 0.25
number of people in the --top-- 64
7  in top   64 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.109375
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.5 0.1
number of people in the --top-- 25
5  in top   25 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.2
great_comorbidity_binary sum 5
great_comorbidity_binary 11 254
on  0.5 0.25
number of people in the --top-- 64
5  in top   64 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.078125
great_comorbidity_binary sum 5
great_comorbidity_binary 11 254
on  0.25 0.1
number of people in the --top-- 25
2  in top   25 total comorbid  10
finished the top top_quarter sum 2
top_quarter percent 0.08
great_co

/tmp/ipykernel_97293/2603949995.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  comorbid_df[newcolname] = comorbidlist
/tmp/ipykernel_97293/2603949995.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  comorbid_df["perfect_"+biasdir+"_comorbidity_"+str(sortgene)+"_ncomorbid"+str(ncomorbid_people)] = np.concatenate((np.repeat(1, ncomorbid_people),np.repeat(0, nnotcomorbid_people)), axis=None)
/tmp/ipykernel_97293/2603949995.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

on  0.75 0.25
number of people in the --top-- 64
157  in top   64 total comorbid  210
nevermind 64  in top   64 total comorbid  210
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 211 254
on  0.5 0.1
number of people in the --top-- 25
105  in top   25 total comorbid  210
nevermind 25  in top   25 total comorbid  210
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 211 254
on  0.5 0.25
number of people in the --top-- 64
105  in top   64 total comorbid  210
nevermind 64  in top   64 total comorbid  210
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 211 254
on  0.25 0.1
number of people in the --top-- 25
52  in top   25 total comorbid  210
nevermind 25  in top   25 total comorbid  210
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_como

...

number of people in the --top-- 25
138  in top   25 total comorbid  185
nevermind 25  in top   25 total comorbid  185
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 186 254
on  0.75 0.25
number of people in the --top-- 64
138  in top   64 total comorbid  185
nevermind 64  in top   64 total comorbid  185
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 186 254
on  0.5 0.1
number of people in the --top-- 25
92  in top   25 total comorbid  185
nevermind 25  in top   25 total comorbid  185
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 186 254
on  0.5 0.25
number of people in the --top-- 64
92  in top   64 total comorbid  185
nevermind 64  in top   64 total comorbid  185
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary

67  in top   25 total comorbid  135
nevermind 25  in top   25 total comorbid  135
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 136 254
on  0.5 0.25
number of people in the --top-- 64
67  in top   64 total comorbid  135
nevermind 64  in top   64 total comorbid  135
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 136 254
on  0.25 0.1
number of people in the --top-- 25
33  in top   25 total comorbid  135
nevermind 25  in top   25 total comorbid  135
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 136 254
on  0.25 0.25
number of people in the --top-- 64
33  in top   64 total comorbid  135
finished the top top_quarter sum 33
top_quarter percent 0.515625
great_comorbidity_binary sum 33
great_comorbidity_binary 136 254
on ncomorbid_people 160
on  0.75 0.1
number of people in the --top--

nevermind 25  in top   25 total comorbid  85
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 86 254
on  0.75 0.25
number of people in the --top-- 64
63  in top   64 total comorbid  85
finished the top top_quarter sum 63
top_quarter percent 0.984375
great_comorbidity_binary sum 63
great_comorbidity_binary 86 254
on  0.5 0.1
number of people in the --top-- 25
42  in top   25 total comorbid  85
nevermind 25  in top   25 total comorbid  85
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 86 254
on  0.5 0.25
number of people in the --top-- 64
42  in top   64 total comorbid  85
finished the top top_quarter sum 42
top_quarter percent 0.65625
great_comorbidity_binary sum 42
great_comorbidity_binary 86 254
on  0.25 0.1
number of people in the --top-- 25
21  in top   25 total comorbid  85
finished the top top_quarter sum 21
top_quarter percent 0.84
great_comorb

number of people in the --top-- 25
26  in top   25 total comorbid  35
nevermind 25  in top   25 total comorbid  35
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 36 254
on  0.75 0.25
number of people in the --top-- 64
26  in top   64 total comorbid  35
finished the top top_quarter sum 26
top_quarter percent 0.40625
great_comorbidity_binary sum 26
great_comorbidity_binary 36 254
on  0.5 0.1
number of people in the --top-- 25
17  in top   25 total comorbid  35
finished the top top_quarter sum 17
top_quarter percent 0.68
great_comorbidity_binary sum 17
great_comorbidity_binary 36 254
on  0.5 0.25
number of people in the --top-- 64
17  in top   64 total comorbid  35
finished the top top_quarter sum 17
top_quarter percent 0.265625
great_comorbidity_binary sum 17
great_comorbidity_binary 36 254
on  0.25 0.1
number of people in the --top-- 25
8  in top   25 total comorbid  35
finished the top top_quarter sum 8
top_quarter p

top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 211 254
on  0.75 0.25
number of people in the --top-- 64
157  in top   64 total comorbid  210
nevermind 64  in top   64 total comorbid  210
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 211 254
on  0.5 0.1
number of people in the --top-- 25
105  in top   25 total comorbid  210
nevermind 25  in top   25 total comorbid  210
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 211 254
on  0.5 0.25
number of people in the --top-- 64
105  in top   64 total comorbid  210
nevermind 64  in top   64 total comorbid  210
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 211 254
on  0.25 0.1
number of people in the --top-- 25
52  in top   25 total comorbid  210
nevermind 25  in top   25 total comorbid  210
finished the 

# work with the dataframes without saveing

In [ ]:
genexpdf, comorbid_df = makegeneexpandcomorbidfile(mean_exp, min_stdev, max_stdev)

on ncomorbid_people 10
on  0.75 0.1
number of people in the --top-- 25
7  in top   25 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.28
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.75 0.25
number of people in the --top-- 64
7  in top   64 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.109375
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.5 0.1
number of people in the --top-- 25
5  in top   25 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.2
great_comorbidity_binary sum 5
great_comorbidity_binary 11 254
on  0.5 0.25
number of people in the --top-- 64
5  in top   64 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.078125
great_comorbidity_binary sum 5
great_comorbidity_binary 11 254
on  0.25 0.1
number of people in the --top-- 25
2  in top   25 total comorbid  10
finished the top top_quarter sum 2
top_quarter percent 0.08
great_co

great_comorbidity_binary 186 254
on ncomorbid_people 210
on  0.75 0.1
number of people in the --top-- 25
157  in top   25 total comorbid  210
nevermind 25  in top   25 total comorbid  210
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 211 254
on  0.75 0.25
number of people in the --top-- 64
157  in top   64 total comorbid  210
nevermind 64  in top   64 total comorbid  210
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 211 254
on  0.5 0.1
number of people in the --top-- 25
105  in top   25 total comorbid  210
nevermind 25  in top   25 total comorbid  210
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 211 254
on  0.5 0.25
number of people in the --top-- 64
105  in top   64 total comorbid  210
nevermind 64  in top   64 total comorbid  210
finished the top top_quarter sum 64
top_quar

on  0.5 0.25
number of people in the --top-- 64
80  in top   64 total comorbid  160
nevermind 64  in top   64 total comorbid  160
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 161 254
on  0.25 0.1
number of people in the --top-- 25
40  in top   25 total comorbid  160
nevermind 25  in top   25 total comorbid  160
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 161 254
on  0.25 0.25
number of people in the --top-- 64
40  in top   64 total comorbid  160
finished the top top_quarter sum 40
top_quarter percent 0.625
great_comorbidity_binary sum 40
great_comorbidity_binary 161 254
on ncomorbid_people 185
on  0.75 0.1
number of people in the --top-- 25
138  in top   25 total comorbid  185
nevermind 25  in top   25 total comorbid  185
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 186 25

top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 111 254
on  0.75 0.25
number of people in the --top-- 64
82  in top   64 total comorbid  110
nevermind 64  in top   64 total comorbid  110
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 111 254
on  0.5 0.1
number of people in the --top-- 25
55  in top   25 total comorbid  110
nevermind 25  in top   25 total comorbid  110
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 111 254
on  0.5 0.25
number of people in the --top-- 64
55  in top   64 total comorbid  110
finished the top top_quarter sum 55
top_quarter percent 0.859375
great_comorbidity_binary sum 55
great_comorbidity_binary 111 254
on  0.25 0.1
number of people in the --top-- 25
27  in top   25 total comorbid  110
nevermind 25  in top   25 total comorbid  110
finished the top top_quarter sum 25
top_quarter percent 1

nevermind 25  in top   25 total comorbid  60
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 61 254
on  0.75 0.25
number of people in the --top-- 64
45  in top   64 total comorbid  60
finished the top top_quarter sum 45
top_quarter percent 0.703125
great_comorbidity_binary sum 45
great_comorbidity_binary 61 254
on  0.5 0.1
number of people in the --top-- 25
30  in top   25 total comorbid  60
nevermind 25  in top   25 total comorbid  60
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 61 254
on  0.5 0.25
number of people in the --top-- 64
30  in top   64 total comorbid  60
finished the top top_quarter sum 30
top_quarter percent 0.46875
great_comorbidity_binary sum 30
great_comorbidity_binary 61 254
on  0.25 0.1
number of people in the --top-- 25
15  in top   25 total comorbid  60
finished the top top_quarter sum 15
top_quarter percent 0.6
great_comorbi

...

nevermind 25  in top   25 total comorbid  60
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 61 254
on  0.75 0.25
number of people in the --top-- 64
45  in top   64 total comorbid  60
finished the top top_quarter sum 45
top_quarter percent 0.703125
great_comorbidity_binary sum 45
great_comorbidity_binary 61 254
on  0.5 0.1
number of people in the --top-- 25
30  in top   25 total comorbid  60
nevermind 25  in top   25 total comorbid  60
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 61 254
on  0.5 0.25
number of people in the --top-- 64
30  in top   64 total comorbid  60
finished the top top_quarter sum 30
top_quarter percent 0.46875
great_comorbidity_binary sum 30
great_comorbidity_binary 61 254
on  0.25 0.1
number of people in the --top-- 25
15  in top   25 total comorbid  60
finished the top top_quarter sum 15
top_quarter percent 0.6
great_comorbi

52  in top   64 total comorbid  210
finished the top top_quarter sum 52
top_quarter percent 0.8125
great_comorbidity_binary sum 52
great_comorbidity_binary 211 254
on ncomorbid_people 10
on  0.75 0.1
number of people in the --top-- 25
7  in top   25 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.28
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.75 0.25
number of people in the --top-- 64
7  in top   64 total comorbid  10
finished the top top_quarter sum 7
top_quarter percent 0.109375
great_comorbidity_binary sum 7
great_comorbidity_binary 11 254
on  0.5 0.1
number of people in the --top-- 25
5  in top   25 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.2
great_comorbidity_binary sum 5
great_comorbidity_binary 11 254
on  0.5 0.25
number of people in the --top-- 64
5  in top   64 total comorbid  10
finished the top top_quarter sum 5
top_quarter percent 0.078125
great_comorbidity_binary sum 5
great_comorbidity_

number of people in the --top-- 64
92  in top   64 total comorbid  185
nevermind 64  in top   64 total comorbid  185
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 186 254
on  0.25 0.1
number of people in the --top-- 25
46  in top   25 total comorbid  185
nevermind 25  in top   25 total comorbid  185
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 186 254
on  0.25 0.25
number of people in the --top-- 64
46  in top   64 total comorbid  185
finished the top top_quarter sum 46
top_quarter percent 0.71875
great_comorbidity_binary sum 46
great_comorbidity_binary 186 254
on ncomorbid_people 210
on  0.75 0.1
number of people in the --top-- 25
157  in top   25 total comorbid  210
nevermind 25  in top   25 total comorbid  210
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 211 254
on  0.75 

finished the top top_quarter sum 33
top_quarter percent 0.515625
great_comorbidity_binary sum 33
great_comorbidity_binary 136 254
on ncomorbid_people 160
on  0.75 0.1
number of people in the --top-- 25
120  in top   25 total comorbid  160
nevermind 25  in top   25 total comorbid  160
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 161 254
on  0.75 0.25
number of people in the --top-- 64
120  in top   64 total comorbid  160
nevermind 64  in top   64 total comorbid  160
finished the top top_quarter sum 64
top_quarter percent 1.0
great_comorbidity_binary sum 64
great_comorbidity_binary 161 254
on  0.5 0.1
number of people in the --top-- 25
80  in top   25 total comorbid  160
nevermind 25  in top   25 total comorbid  160
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 161 254
on  0.5 0.25
number of people in the --top-- 64
80  in top   64 total comorbid 

nevermind 25  in top   25 total comorbid  85
finished the top top_quarter sum 25
top_quarter percent 1.0
great_comorbidity_binary sum 25
great_comorbidity_binary 86 254
on  0.5 0.25
number of people in the --top-- 64
42  in top   64 total comorbid  85
finished the top top_quarter sum 42
top_quarter percent 0.65625
great_comorbidity_binary sum 42
great_comorbidity_binary 86 254
on  0.25 0.1
number of people in the --top-- 25
21  in top   25 total comorbid  85
finished the top top_quarter sum 21
top_quarter percent 0.84
great_comorbidity_binary sum 21
great_comorbidity_binary 86 254
on  0.25 0.25
number of people in the --top-- 64
21  in top   64 total comorbid  85
finished the top top_quarter sum 21
top_quarter percent 0.328125
great_comorbidity_binary sum 21
great_comorbidity_binary 86 254
on ncomorbid_people 110
on  0.75 0.1
number of people in the --top-- 25
82  in top   25 total comorbid  110
nevermind 25  in top   25 total comorbid  110
finished the top top_quarter sum 25
top_quart

In [ ]:
genexpdf

,Patient,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9
60,Shelagh,98.779218,98.708682,99.468340,99.463126,98.712805,95.976763,101.757957,87.253219,75.075929
228,Janis,100.006326,98.527377,101.359444,102.591400,93.148773,90.349025,95.654498,95.202761,77.786986
53,Romonda,99.796200,97.466811,101.522911,97.829497,101.285180,89.958043,102.522378,97.532943,78.257896
100,Naoma,101.736388,96.545187,100.701240,104.057672,93.700718,104.570180,96.832408,94.958720,79.470634
14,Carolyn,101.062419,102.610926,98.025459,100.683183,92.399670,101.318058,101.586934,93.946687,79.753531
...,...,...,...,...,...,...,...,...,...,...
158,Nollie,99.049762,95.331853,102.080694,89.414176,106.688925,90.094271,93.915500,91.859135,118.559196
163,Jenifer,99.561108,103.032042,102.653895,98.016907,105.449068,101.157244,102.787660,109.430842,119.589138
178,Orelia,99.231241,100.425252,100.985185,103.026161,98.926887,109.551725,109.538573,86.763492,122.556934
148,Fania,101.193140,101.562787,103.047781,104.331557,107.761405,103.271833,96.703853,97.201772,123.061706


In [ ]:
comorbid_df

,Patient,perfect_top_comorbidity_gene1_ncomorbid10,great_comorbidity_gene1_ncomorbid10_top25_topcomorbid7,random_comorbidity_ncomorbid_10random_1,great_comorbidity_gene1_ncomorbid10_top64_topcomorbid7,random_comorbidity_ncomorbid_10random_2,great_comorbidity_gene1_ncomorbid10_top25_topcomorbid5,random_comorbidity_ncomorbid_10random_3,great_comorbidity_gene1_ncomorbid10_top64_topcomorbid5,random_comorbidity_ncomorbid_10random_4,...,great_comorbidity_gene9_ncomorbid160_bottom64_topcomorbid64,great_comorbidity_gene9_ncomorbid160_bottom64_topcomorbid40,perfect_bottom_comorbidity_gene9_ncomorbid185,great_comorbidity_gene9_ncomorbid185_bottom25_topcomorbid25,great_comorbidity_gene9_ncomorbid185_bottom64_topcomorbid64,great_comorbidity_gene9_ncomorbid185_bottom64_topcomorbid46,perfect_bottom_comorbidity_gene9_ncomorbid210,great_comorbidity_gene9_ncomorbid210_bottom25_topcomorbid25,great_comorbidity_gene9_ncomorbid210_bottom64_topcomorbid64,great_comorbidity_gene9_ncomorbid210_bottom64_topcomorbid52
60,Shelagh,0,0,0,0,0,0,0,0,1,...,1,0,1,1,1,1,1,1,1,1
228,Janis,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,0,1,1,1,1
53,Romonda,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
100,Naoma,1,0,0,0,0,0,0,0,0,...,1,0,1,1,1,0,1,1,1,1
14,Carolyn,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,Nollie,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,1,1
163,Jenifer,0,0,0,0,0,1,0,0,0,...,1,1,0,1,1,1,0,1,1,1
178,Orelia,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,1,0,0,0,1
148,Fania,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,0,1,1,1


In [ ]:
testdf = comorbid_df.merge(genexpdf, on="Patient")
testdf = testdf.sort_values("gene1", ascending=False)
testdf

,Patient,perfect_top_comorbidity_gene1_ncomorbid10,great_comorbidity_gene1_ncomorbid10_top25_topcomorbid7,random_comorbidity_ncomorbid_10random_1,great_comorbidity_gene1_ncomorbid10_top64_topcomorbid7,random_comorbidity_ncomorbid_10random_2,great_comorbidity_gene1_ncomorbid10_top25_topcomorbid5,random_comorbidity_ncomorbid_10random_3,great_comorbidity_gene1_ncomorbid10_top64_topcomorbid5,random_comorbidity_ncomorbid_10random_4,...,great_comorbidity_gene9_ncomorbid210_bottom64_topcomorbid52,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9
217,Bill,1,1,0,1,0,0,0,0,0,...,1,102.338596,99.396148,101.506159,104.736816,106.843830,93.611052,107.668635,99.748479,109.278132
6,Benedetta,1,0,0,0,0,0,0,0,0,...,1,102.224527,99.929575,95.547790,99.595978,99.567295,97.011558,91.916406,88.853783,81.943887
13,Dee Dee,1,1,0,0,0,0,0,0,0,...,1,102.078581,101.236611,95.683727,103.091150,87.521532,105.731873,102.659474,96.498945,86.039396
234,Serene,1,1,0,0,0,0,0,0,0,...,1,102.043110,96.946302,99.172339,95.693372,91.517655,102.711105,109.197989,103.808894,113.626849
238,Joey,1,0,0,0,0,1,0,0,1,...,1,102.017568,100.002254,97.280946,104.720920,96.874754,94.244268,103.823448,103.763106,114.799074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Livvie,0,0,0,0,0,0,0,0,0,...,1,97.968462,104.514657,100.654563,96.946865,103.410189,99.159430,106.430008,123.832155,97.878193
223,Emeline,0,0,0,0,0,0,0,0,0,...,1,97.930453,102.137616,96.221759,99.971045,107.354475,99.030273,89.754204,104.514551,110.607007
205,Hedvig,0,0,0,0,0,0,0,0,0,...,1,97.656803,100.686174,99.099925,97.271943,101.571344,92.983283,94.770397,88.896794,108.123428
27,Lorrin,0,0,0,0,0,0,0,1,1,...,1,97.462229,101.235493,98.590141,100.647488,97.445956,106.914460,88.452207,85.599374,89.939607


In [ ]:
comorbid_df.columns

Index(['Patient', 'perfect_top_comorbidity_gene1_ncomorbid10',
       'great_comorbidity_gene1_ncomorbid10_top25_topcomorbid7',
       'random_comorbidity_ncomorbid_10random_1',
       'great_comorbidity_gene1_ncomorbid10_top64_topcomorbid7',
       'random_comorbidity_ncomorbid_10random_2',
       'great_comorbidity_gene1_ncomorbid10_top25_topcomorbid5',
       'random_comorbidity_ncomorbid_10random_3',
       'great_comorbidity_gene1_ncomorbid10_top64_topcomorbid5',
       'random_comorbidity_ncomorbid_10random_4',
       ...
       'great_comorbidity_gene9_ncomorbid160_bottom64_topcomorbid64',
       'great_comorbidity_gene9_ncomorbid160_bottom64_topcomorbid40',
       'perfect_bottom_comorbidity_gene9_ncomorbid185',
       'great_comorbidity_gene9_ncomorbid185_bottom25_topcomorbid25',
       'great_comorbidity_gene9_ncomorbid185_bottom64_topcomorbid64',
       'great_comorbidity_gene9_ncomorbid185_bottom64_topcomorbid46',
       'perfect_bottom_comorbidity_gene9_ncomorbid210',
    

In [ ]:
onecol="great_comorbidity_gene1_ncomorbid10_bottom25_topcomorbid7"
list(testdf[onecol])
#list(np.cumsum(testdf[onecol]))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [ ]:
comorbid_df["great_comorbidity_gene1_ncomorbid160_top64_topcomorbid40"].sum()

In [ ]:
a= [1,0,1,0,1]

In [ ]:
a.index(0)